# ANYmal C Robot Locomotion Simulation
This notebook demonstrates how to simulate the ANYmal C quadruped robot's trotting motion using MuJoCo.

![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)


This notebook was written by [Ragib_Rownak](https://github.com/RagibRownak/Anymal_c_locomotion_rl).

In [1]:
# Import Required Libraries
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import Video, display, HTML
import mediapy as media
import time
import matplotlib.animation as animation
from matplotlib import rc
from IPython.core.display import display as ipydisplay

# Try to import plotly for 3D visualization
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    has_plotly = True
except ImportError:
    print("Installing plotly for 3D visualization...")
    os.system("pip install plotly kaleido")  # kaleido for static image export
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        has_plotly = True
    except ImportError:
        has_plotly = False
        print("Failed to install plotly. Skipping 3D visualization.")

# Try to import the viewer if available
try:
    from mujoco import viewer
    has_viewer = True
except ImportError:
    print("Couldn't import the MuJoCo viewer module")
    has_viewer = False

/home/ragib/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/tmp/ipykernel_17209/2737128555.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display as ipydisplay


## Load the ANYmal C Model
Load the ANYmal C robot model from its XML file.

In [2]:
# Path to the model XML file
model_path = 'scene.xml'  # Using scene.xml instead of anymal_c.xml to get the textured ground

# Load the model
model = mujoco.MjModel.from_xml_path(model_path)
data = mujoco.MjData(model)

# Initialize the robot to a standing position
mujoco.mj_resetData(model, data)

# Get information about the physics timestep
timestep = model.opt.timestep
print(f'Model loaded successfully with {model.nq} position DoFs and {model.nu} actuated DoFs')
print(f'Physics timestep: {timestep:.4f} seconds')

Model loaded successfully with 19 position DoFs and 12 actuated DoFs
Physics timestep: 0.0020 seconds


## Define Kinematic Functions
Define functions to generate trotting kinematics for the robot's legs.

In [3]:
def cos_wave(t, step_period, scale):
    # Matching the JAX implementation exactly
    _cos_wave = -np.cos(((2*np.pi)/step_period)*t)
    return _cos_wave * (scale/2) + (scale/2)

def dcos_wave(t, step_period, scale):
    """
    Derivative of the cos wave, for reference velocity
    """
    # Matching the JAX implementation exactly
    return ((scale*np.pi) / step_period) * np.sin(((2*np.pi)/step_period)*t)

def quintic_spline(t, t0, tf, x0, xf, v0=0, vf=0, a0=0, af=0):
    """
    Generate a smooth quintic spline trajectory between two points.
    Ensures continuous position, velocity, and acceleration.
    
    Args:
        t: Current time
        t0: Start time
        tf: End time
        x0: Start position
        xf: End position
        v0: Start velocity (default 0)
        vf: End velocity (default 0)
        a0: Start acceleration (default 0)
        af: End acceleration (default 0)
    
    Returns:
        Position at time t
    """
    # Normalized time from 0 to 1
    if tf == t0:
        # Handle edge case
        return x0
        
    s = (t - t0) / (tf - t0)
    
    # Clamp s to [0, 1] range
    s = max(0, min(1, s))
    
    # Quintic polynomial coefficients
    h = xf - x0
    a = 6*h - 3*(vf+v0)*(tf-t0) + 0.5*(af-a0)*(tf-t0)**2
    b = -15*h + 7*v0*(tf-t0) + 8*vf*(tf-t0) - a0*(tf-t0)**2 - 1.5*af*(tf-t0)**2
    c = 10*h - 4*(vf+v0)*(tf-t0) + 0.5*(af+a0)*(tf-t0)**2
    
    # Evaluate quintic polynomial
    position = x0 + s**3 * (a + s * (b + s * c))
    return position

def filter_signal(signal, alpha=0.5):
    """
    Apply a simple low-pass filter to smooth the signal.
    alpha is the filter coefficient (0 < alpha < 1).
    Lower alpha means more smoothing.
    """
    filtered = np.zeros_like(signal)
    filtered[0] = signal[0]
    
    for i in range(1, len(signal)):
        filtered[i] = alpha * signal[i] + (1 - alpha) * filtered[i-1]
    
    return filtered
# Define a low-pass filter for smoothing control signals
def low_pass_filter(new_value, old_value, alpha=0.1):  # Default alpha for moderate smoothing
    """
    Simple first-order low-pass filter. Lower alpha means more smoothing.
    """
    return alpha * new_value + (1 - alpha) * old_value

def make_kinematic_ref(sinusoid, step_k, scale=0.3, dt=None, smooth=True, filter_strength=0.2):
    """
    Makes trotting kinematics for the 12 leg joints.
    Closely follows the JAX implementation in the prompt.
    step_k is the number of timesteps it takes to raise and lower a given foot.
    A gait cycle is 2 * step_k * dt seconds long.
    dt should match the MuJoCo timestep for accurate simulation.
    
    Args:
        sinusoid: Function that generates base motion
        step_k: Steps to complete one leg motion
        scale: Amplitude of the motion
        dt: Timestep duration
        smooth: Whether to use additional smoothing
        filter_strength: Amount of filtering to apply (0-1)
    """
    if dt is None:
        dt = model.opt.timestep  # Use MuJoCo's default timestep if none provided

    # Generate time steps exactly as in JAX implementation
    _steps = np.arange(step_k)
    step_period = step_k * dt
    t = _steps * dt

    # Get base sinusoidal trajectory
    wave = sinusoid(t, step_period, scale)
    
    # Commands for one step of an active front leg
    fleg_cmd_block = np.concatenate(
        [np.zeros((step_k, 1)),
        wave.reshape(step_k, 1),
        -2*wave.reshape(step_k, 1)],
        axis=1
    )
    # Our standing config reverses front and hind legs, matching JAX implementation
    h_leg_cmd_block = -1 * fleg_cmd_block

    # Match JAX implementation block structure exactly
    block1 = np.concatenate([
        np.zeros((step_k, 3)),
        fleg_cmd_block,
        h_leg_cmd_block,
        np.zeros((step_k, 3))],
        axis=1
    )

    block2 = np.concatenate([
        fleg_cmd_block,
        np.zeros((step_k, 3)),
        np.zeros((step_k, 3)),
        h_leg_cmd_block],
        axis=1
    )
    # In one step cycle, both pairs of active legs have inactive and active phases
    step_cycle = np.concatenate([block1, block2], axis=0)
    
    # Apply smoothing if requested
    if smooth:
        # Apply quintic spline smoothing at transition points
        smoothed_cycle = np.copy(step_cycle)
        transition_width = max(3, step_k // 8)  # Width of transition smoothing
        
        # Process each joint separately
        for joint in range(step_cycle.shape[1]):
            # Find significant transitions (changes > 1% of max range)
            joint_traj = step_cycle[:, joint]
            range_threshold = 0.01 * (np.max(joint_traj) - np.min(joint_traj))
            
            # Calculate absolute differences between consecutive points
            diffs = np.abs(np.diff(joint_traj))
            # Find transition points (where difference exceeds threshold)
            transition_indices = np.where(diffs > range_threshold)[0]
            
            # Apply smoothing around each transition point
            for idx in transition_indices:
                # Define smoothing region
                start_idx = max(0, idx - transition_width)
                end_idx = min(len(joint_traj) - 1, idx + transition_width + 1)
                
                # Apply quintic spline between start and end of transition
                x0 = joint_traj[start_idx]
                xf = joint_traj[end_idx]
                t0 = start_idx * dt
                tf = end_idx * dt
                
                # Calculate velocities for better continuity
                if start_idx > 0 and end_idx < len(joint_traj) - 1:
                    v0 = (joint_traj[start_idx] - joint_traj[start_idx-1]) / dt
                    vf = (joint_traj[end_idx+1] - joint_traj[end_idx]) / dt
                else:
                    v0 = 0
                    vf = 0
                
                # Replace trajectory with smoothed version
                for j in range(start_idx, end_idx + 1):
                    smoothed_cycle[j, joint] = quintic_spline(
                        j * dt, t0, tf, x0, xf, v0, vf
                    )
        
        # Apply additional filtering for smoother transitions
        if filter_strength > 0:
            # Apply filter to each joint's trajectory
            for i in range(smoothed_cycle.shape[1]):
                smoothed_cycle[:, i] = filter_signal(smoothed_cycle[:, i], 1.0 - filter_strength)
        
        return smoothed_cycle
    else:
        # Return unsmoothed cycle
        return step_cycle

# Add this quaternion to Euler angles conversion function
def compute_rpy_from_quaternion(quat):
    """
    Convert quaternion to roll, pitch, yaw angles.
    
    Args:
        quat: Quaternion in [w, x, y, z] format
        
    Returns:
        Tuple of (roll, pitch, yaw) angles in radians
    """
    # Ensure quaternion is normalized
    quat = quat / np.linalg.norm(quat)
    
    # Extract components for readability
    w, x, y, z = quat
    
    # Roll (rotation around X-axis)
    sinr_cosp = 2.0 * (w * x + y * z)
    cosr_cosp = 1.0 - 2.0 * (x * x + y * y)
    roll = np.arctan2(sinr_cosp, cosr_cosp)
    
    # Pitch (rotation around Y-axis)
    sinp = 2.0 * (w * y - z * x)
    # Check for gimbal lock
    if abs(sinp) >= 1:
        pitch = np.copysign(np.pi / 2, sinp)  # Use 90 degrees if in gimbal lock
    else:
        pitch = np.arcsin(sinp)
    
    # Yaw (rotation around Z-axis)
    siny_cosp = 2.0 * (w * z + x * y)
    cosy_cosp = 1.0 - 2.0 * (y * y + z * z)
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    
    return (roll, pitch, yaw)

class ImprovedPDController:
    """
    Enhanced PD controller with feed-forward component and adaptive gains.
    Provides smoother control and better tracking performance.
    """
    def __init__(self, kp=300.0, kd=50.0, kff=0.5, max_velocity=0.1): # Reduced max_velocity, adjusted gains
        # Controller gains
        self.kp = kp  # Position gain - adjusted for stability
        self.kd = kd  # Velocity gain - increased damping
        self.kff = kff  # Feed-forward gain - significantly reduced for smoothness
        
        # Control limits
        self.max_velocity = max_velocity  # Significantly lower max velocity for very smooth movement
        self.max_accel = 10.0  # Reduced acceleration limit for smoother movement
        
        # State tracking
        self.prev_targets = None
        self.prev_prev_targets = None
        self.prev_error = None
        self.prev_control = None
        self.dt = None
        
        # Adaptive gain scaling factors
        self.error_integration = np.zeros(12)  # For 12 joints
        self.ki = 0.01  # Very low integral gain for minimal drift correction
        self.integral_windup_limit = 5.0  # Further reduced limit
        
    def set_dt(self, dt):
        """Set controller timestep"""
        self.dt = dt
        
    def reset(self):
        """Reset controller state"""
        self.prev_targets = None
        self.prev_prev_targets = None
        self.prev_error = None
        self.prev_control = None
        self.error_integration = np.zeros(12)
        
    def compute_control(self, targets, current_positions, current_velocities):
        """Compute enhanced PD control signals with feed-forward component.

        Args:
            targets: Target joint positions
            current_positions: Current joint positions
            current_velocities: Current joint velocities

        Returns:
            Control signals to apply
        """
        if self.dt is None:
            raise ValueError("Controller timestep not set. Call set_dt() first.")
            
        # Calculate position error
        error = targets - current_positions
        
        # Update integral term with anti-windup
        self.error_integration += error * self.dt
        self.error_integration = np.clip(
            self.error_integration, 
            -self.integral_windup_limit, 
            self.integral_windup_limit
        )
        
        # Feed-forward velocity and acceleration components
        if self.prev_targets is not None:
            # Target velocity using finite difference
            target_velocity = (targets - self.prev_targets) / self.dt
            target_velocity = np.clip(target_velocity, -self.max_velocity, self.max_velocity)
            
            # Feed-forward acceleration if we have enough history
            if self.prev_prev_targets is not None:
                prev_velocity = (self.prev_targets - self.prev_prev_targets) / self.dt
                target_accel = (target_velocity - prev_velocity) / self.dt
                target_accel = np.clip(target_accel, -self.max_accel, self.max_accel)
            else:
                target_accel = np.zeros_like(targets)
        else:
            # Initial conditions
            target_velocity = np.zeros_like(targets)
            target_accel = np.zeros_like(targets)
        
        # Velocity error (actual vs desired velocity)
        velocity_error = target_velocity - current_velocities
        
        # PID control law with feed-forward
        control = (self.kp * error + 
                  self.kd * velocity_error + 
                  self.ki * self.error_integration + 
                  self.kff * target_velocity)
        
        # Rate limiting for smoother control transitions
        if self.prev_control is not None:
            max_change = 10.0 * self.dt  # Significantly reduced rate of change for smoother control
            control_delta = control - self.prev_control
            limited_delta = np.clip(control_delta, -max_change, max_change)
            control = self.prev_control + limited_delta
        
        # Store values for next iteration
        self.prev_prev_targets = self.prev_targets.copy() if self.prev_targets is not None else None
        self.prev_targets = targets.copy()
        self.prev_control = control.copy()
        self.prev_error = error
        
        return control

# Also update the legacy PDController class
class PDController:
    """
    PD controller for smoother joint control.
    Applies proportional and derivative control to reduce oscillation.
    """
    def __init__(self, kp=150.0, kd=20.0, max_velocity=0.1):  # Reduced max_velocity
        self.kp = kp  # Proportional gain
        self.kd = kd  # Derivative gain
        self.max_velocity = max_velocity  # Maximum velocity limit (reduced)
        self.prev_targets = None
        self.prev_error = None
        self.target_velocity = None
        self.dt = None

    def set_dt(self, dt):
        """Set controller timestep"""
        self.dt = dt
        
    def compute_control(self, targets, current_positions, current_velocities):
        """
        Compute PD control signals.
        
        Args:
            targets: Target joint positions
            current_positions: Current joint positions
            current_velocities: Current joint velocities
        
        Returns:
            Control signals to apply
        """
        if self.dt is None:
            raise ValueError("Controller timestep not set. Call set_dt() first.")
            
        # Calculate position error
        error = targets - current_positions
        
        # Calculate target velocities using finite difference if we have previous targets
        if self.prev_targets is not None:
            # Calculate target velocities with limiting
            raw_velocity = (targets - self.prev_targets) / self.dt
            target_velocity = np.clip(raw_velocity, -self.max_velocity, self.max_velocity)
        else:
            # No velocity on first step
            target_velocity = np.zeros_like(targets)
            
        # Velocity error
        velocity_error = target_velocity - current_velocities
        
        # PD control law
        control = self.kp * error + self.kd * velocity_error
        
        # Store values for next iteration
        self.prev_targets = targets.copy()
        self.prev_error = error
        
        return control

# Simulation parameters
fps = 30
duration = 5  # seconds
physics_steps_per_frame = max(1, int(1.0 / (fps * model.opt.timestep)))
total_physics_steps = int(duration / model.opt.timestep)
n_frames = total_physics_steps // physics_steps_per_frame

# Display configuration
height = 480
width = 640

print(f"Physics timestep: {model.opt.timestep:.6f} seconds")
print(f"Total frames to render: {n_frames}")

# Reset simulation fully
mujoco.mj_resetData(model, data)

# Create a new renderer
if 'renderer' in globals():
    del renderer
renderer = mujoco.Renderer(model, height=height, width=width)
video_frames = []

# Get joint actuator IDs for the robot's legs
actuator_names = [model.actuator(i).name for i in range(model.nu)]
print(f"Available actuators: {actuator_names}")

# BALANCE CONTROLLER CONFIGURATION
# Optimized for extremely stable standing with minimal oscillation
controller = ImprovedPDController(kp=350.0, kd=40.0, kff=2.0, max_velocity=1.0)
controller.set_dt(model.opt.timestep)
controller.reset()

# Define a stable stance posture
# This creates a wide, low stance for maximum stability
stable_stance = np.zeros(12)

# Wider stance with outward hip angles for stability
stable_stance[0] = 0.08   # LF_HAA - outward angle (further increased)
stable_stance[3] = -0.08  # RF_HAA - outward angle (further increased)
stable_stance[6] = 0.08   # LH_HAA - outward angle (further increased)
stable_stance[9] = -0.08  # RH_HAA - outward angle (further increased)

# Lower center of mass with bent knees
stable_stance[2] = 0.4    # LF_KFE - knee bend (further increased)
stable_stance[5] = 0.4    # RF_KFE - knee bend (further increased)
stable_stance[8] = 0.4    # LH_KFE - knee bend (further increased)
stable_stance[11] = 0.4   # RH_KFE - knee bend (further increased)

# Add slight offset to hip flexion/extension for weight distribution
stable_stance[1] = 0.05   # LF_HFE - slight forward lean
stable_stance[4] = 0.05   # RF_HFE - slight forward lean
stable_stance[7] = -0.05  # LH_HFE - slight backward lean
stable_stance[10] = -0.05 # RH_HFE - slight backward lean

# Create reference positions array (all set to stable stance)
reference_positions = np.tile(stable_stance, (total_physics_steps, 1))

# Gentle transition from current pose to stable stance
initial_pose = data.qpos[7:19].copy()
transition_steps = int(1.5 / model.opt.timestep)  # 1.5 second transition (increased for smoother movement)

for i in range(min(transition_steps, total_physics_steps)):
    # Smooth transition using cubic ease-in-out
    progress = i / transition_steps
    # Smooth sigmoid-like function: 3*t^2 - 2*t^3
    smooth_factor = 3 * (progress**2) - 2 * (progress**3)
    reference_positions[i] = initial_pose + smooth_factor * (stable_stance - initial_pose)

# Apply additional smoothing to the entire trajectory
filter_alpha = 0.3  # Lower value means more smoothing
for joint in range(reference_positions.shape[1]):
    reference_positions[:, joint] = filter_signal(reference_positions[:, joint], filter_alpha)

# Data collection arrays
position_history = []
velocity_history = []
control_history = []
com_positions = []  # For tracking center of mass

# Balance regulation strength
balance_gain = 2.0  # Further increased for stronger balance response

# Run simulation with balance regulation
frame_count = 0
prev_control = np.zeros(12)  # For control smoothing

for i in range(total_physics_steps):
    # Get current joint positions and velocities
    current_positions = data.qpos[7:19]
    current_velocities = data.qvel[6:18]
    
    # Get reference position for current step
    targets = reference_positions[i]
    
    # Get body orientation and compute orientation error
    orientation = data.qpos[3:7]  # Quaternion for base orientation
    
    # Optional: Compute center of mass
    # This requires computing the mass-weighted position of all bodies
    # For simplicity, we'll use the base position as a proxy
    base_position = data.qpos[:3]
    com_positions.append(base_position.copy())
    
    # Compute balance adjustment - stiffen joints when tilted
    # For simplicity, we use a proxy - just check if base is level
    # A more sophisticated approach would compute the full COM projection
    roll, pitch = compute_rpy_from_quaternion(orientation)[0:2]
    
    # Increase stiffness if robot is tilting
    tilt_amount = np.sqrt(roll**2 + pitch**2)
    stiffness_factor = 1.0 + balance_gain * tilt_amount
    
    # Apply orientation adjustment to controller
    controller.kp = 350.0 * stiffness_factor  # Increase stiffness when tilted (matched to init value)
    
    # Compute control signals 
    control_signals = controller.compute_control(targets, current_positions, current_velocities)
    
    # Rate limit control signals for smoother transitions
    if i > 0:
        max_change = 0.05  # Further reduced limit for even smoother transitions
        control_signals = np.clip(control_signals, prev_control - max_change, prev_control + max_change)
    
    # Store for next iteration
    prev_control = control_signals.copy()
    
    # Apply control signals to actuators
    data.ctrl[:] = control_signals
    
    # Store data for analysis
    position_history.append(current_positions.copy())
    velocity_history.append(current_velocities.copy())
    control_history.append(control_signals.copy())
    
    # Step the simulation
    mujoco.mj_step(model, data)
    
    # Only render every physics_steps_per_frame steps
    if i % physics_steps_per_frame == 0:
        renderer.update_scene(data, camera=-1)
        pixels = renderer.render()
        video_frames.append(pixels.copy())
        frame_count += 1
        
        if frame_count % 20 == 0:
            print(f"Rendered {frame_count}/{n_frames} frames")

print(f"Total frames rendered: {len(video_frames)}")

# Display the video
media.show_video(video_frames, fps=fps)

print("Balance controller simulation complete!")
print(f"Max joint velocity: {np.max(np.abs(velocity_history)):.4f} rad/s")

Physics timestep: 0.002000 seconds
Total frames to render: 156
Available actuators: ['LF_HAA', 'LF_HFE', 'LF_KFE', 'RF_HAA', 'RF_HFE', 'RF_KFE', 'LH_HAA', 'LH_HFE', 'LH_KFE', 'RH_HAA', 'RH_HFE', 'RH_KFE']
Rendered 20/156 frames
Rendered 40/156 frames
Rendered 60/156 frames
Rendered 80/156 frames
Rendered 100/156 frames
Rendered 120/156 frames
Rendered 140/156 frames
Total frames rendered: 157


This browser does not support the video tag.


Balance controller simulation complete!
Max joint velocity: 1.0588 rad/s


## Analyze Robot Performance

Let's analyze the robot's motion by plotting its center of mass trajectory.

In [4]:
# Reset simulation to gather trajectory data
mujoco.mj_resetData(model, data)

# Arrays to store center of mass position
com_positions = []

# Create command list using our improved controller
commands = []
controller = ImprovedPDController(kp=180.0, kd=20.0, kff=25.0)
controller.set_dt(model.opt.timestep)

# Generate smoother reference trajectory
step_k = 25  # Steps to raise and lower a foot
poses = make_kinematic_ref(cos_wave, step_k=step_k, dt=model.opt.timestep, smooth=True, filter_strength=0.2)

# Generate command list (precompute for consistency)
command_steps = n_frames * physics_steps_per_frame
n_cycles = (command_steps // len(poses)) + 1
reference_positions = np.tile(poses, (n_cycles, 1))[:command_steps]

# Run simulation to collect data
for i in range(command_steps):
    # Apply control or store command for later use
    if i < len(reference_positions):
        targets = reference_positions[i]
        # Initially use zero velocities for command generation
        if i == 0:
            commands.append(targets.copy())
        else:
            # Get current joint positions and velocities (dummy at first)
            current_positions = np.zeros_like(targets) if i == 0 else data.qpos[7:19]
            current_velocities = np.zeros_like(targets) if i == 0 else data.qvel[6:18]
            control = controller.compute_control(targets, current_positions, current_velocities)
            commands.append(control.copy())
    else:
        # Just repeat the last command if we run out
        commands.append(commands[-1].copy())
    
    # Apply control
    if i < len(commands):
        data.ctrl[:] = commands[i]
    
    # Step the simulation
    mujoco.mj_step(model, data)
    
    # Store center of mass position at frame intervals
    if i % physics_steps_per_frame == 0 and len(com_positions) < n_frames:
        com_positions.append(data.qpos[:3].copy())  # First 3 positions are x, y, z coordinates

# Convert to numpy array
com_positions = np.array(com_positions)

# Use Plotly for 3D visualization
if has_plotly:
    # Following the approach from anymal_c_kinematics.py
    fig = go.Figure()
    
    # Add trajectory line
    fig.add_trace(go.Scatter3d(
        x=com_positions[:, 0],
        y=com_positions[:, 1],
        z=com_positions[:, 2],
        mode='lines',
        line=dict(color='blue', width=5),
        name="Center of Mass"
    ))
    
    # Add start point
    fig.add_trace(go.Scatter3d(
        x=[com_positions[0, 0]],
        y=[com_positions[0, 1]],
        z=[com_positions[0, 2]],
        mode='markers',
        marker=dict(color='green', size=10, symbol='circle'),
        name="Start"
    ))
    
    # Add end point
    fig.add_trace(go.Scatter3d(
        x=[com_positions[-1, 0]],
        y=[com_positions[-1, 1]],
        z=[com_positions[-1, 2]],
        mode='markers',
        marker=dict(color='red', size=10, symbol='x'),
        name="End"
    ))
    
    # Update layout
    fig.update_layout(
        title="Center of Mass Trajectory",
        scene=dict(
            xaxis_title="X Position (m)",
            yaxis_title="Y Position (m)",
            zaxis_title="Z Position (m)",
            aspectmode='data'
        ),
        width=800,
        height=700
    )
    
    # Add fallback for environments where interactive Plotly doesn't work
    try:
        fig.show()
    except Exception as e:
        print(f"Interactive plotting error: {e}")
        print("Falling back to matplotlib for basic visualization")
        
        # Create a matplotlib figure for fallback
        plt.figure(figsize=(10, 8))
        
        # Plot X-Y trajectory (top view)
        plt.subplot(2, 1, 1)
        plt.plot(com_positions[:, 0], com_positions[:, 1])
        plt.scatter(com_positions[0, 0], com_positions[0, 1], color='green', s=100, marker='o', label='Start')
        plt.scatter(com_positions[-1, 0], com_positions[-1, 1], color='red', s=100, marker='x', label='End')
        plt.title('Top View (X-Y)')
        plt.xlabel('X Position (m)')
        plt.ylabel('Y Position (m)')
        plt.grid(True)
        plt.legend()
        
        # Plot Z over time (height)
        plt.subplot(2, 1, 2)
        plt.plot(np.arange(len(com_positions)), com_positions[:, 2])
        plt.title('Robot Height vs Time')
        plt.xlabel('Time Step')
        plt.ylabel('Z Position (m)')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
else:
    # If Plotly is not available, use matplotlib for basic visualization
    plt.figure(figsize=(10, 8))
    
    # Plot X-Y trajectory (top view)
    plt.subplot(2, 1, 1)
    plt.plot(com_positions[:, 0], com_positions[:, 1])
    plt.scatter(com_positions[0, 0], com_positions[0, 1], color='green', s=100, marker='o', label='Start')
    plt.scatter(com_positions[-1, 0], com_positions[-1, 1], color='red', s=100, marker='x', label='End')
    plt.title('Top View (X-Y)')
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.grid(True)
    plt.legend()
    
    # Plot Z over time (height)
    plt.subplot(2, 1, 2)
    plt.plot(np.arange(len(com_positions)), com_positions[:, 2])
    plt.title('Robot Height vs Time')
    plt.xlabel('Time Step')
    plt.ylabel('Z Position (m)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

## Examine Model Structure
Let's examine the structure of the ANYmal C model to better understand the robot.

In [5]:
# Examine the structure of the ANYmal C model
print("Joints:")
for i in range(model.njnt):
    print(f"  {i}: {model.joint(i).name}")

print("\nBodies:")
for i in range(model.nbody):
    print(f"  {i}: {model.body(i).name}")

print("\nActuators:")
for i in range(model.nu):
    print(f"  {i}: {model.actuator(i).name}")

Joints:
  0: 
  1: LF_HAA
  2: LF_HFE
  3: LF_KFE
  4: RF_HAA
  5: RF_HFE
  6: RF_KFE
  7: LH_HAA
  8: LH_HFE
  9: LH_KFE
  10: RH_HAA
  11: RH_HFE
  12: RH_KFE

Bodies:
  0: world
  1: base
  2: LF_HIP
  3: LF_THIGH
  4: LF_SHANK
  5: RF_HIP
  6: RF_THIGH
  7: RF_SHANK
  8: LH_HIP
  9: LH_THIGH
  10: LH_SHANK
  11: RH_HIP
  12: RH_THIGH
  13: RH_SHANK

Actuators:
  0: LF_HAA
  1: LF_HFE
  2: LF_KFE
  3: RF_HAA
  4: RF_HFE
  5: RF_KFE
  6: LH_HAA
  7: LH_HFE
  8: LH_KFE
  9: RH_HAA
  10: RH_HFE
  11: RH_KFE


## Try Different Gaits
Let's implement a function to try different gaits and compare their performance.

In [6]:
def try_different_gaits(model, data, renderer, use_improved_control=True):
    # Parameters to experiment with
    gait_params = [
        {'name': 'Trot', 'step_k': 25, 'scale': 0.2},
        {'name': 'Pace', 'step_k': 35, 'scale': 0.25},
        {'name': 'Bound', 'step_k': 30, 'scale': 0.3}
    ]
    
    all_videos = []
    
    # Physics timestep and simulation parameters
    dt = model.opt.timestep
    fps = 30
    physics_steps_per_frame = max(1, int(1.0 / (fps * dt)))
    
    # Duration per gait
    seconds_per_gait = 3
    total_physics_steps_per_gait = int(seconds_per_gait / dt)
    
    # Initialize controller based on selection
    if use_improved_control:
        controller = ImprovedPDController(kp=180.0, kd=20.0, kff=25.0, max_velocity=8.0)
    else:
        controller = PDController(kp=150.0, kd=15.0, max_velocity=8.0)
    
    controller.set_dt(dt)
    
    for params in gait_params:
        print(f"Simulating {params['name']} gait...")
        # Reset the simulation
        mujoco.mj_resetData(model, data)
        
        # Reset controller state for new gait
        if use_improved_control:
            controller = ImprovedPDController(kp=180.0, kd=20.0, kff=25.0, max_velocity=8.0)
        else:
            controller = PDController(kp=150.0, kd=15.0, max_velocity=8.0)
            
        controller.set_dt(dt)
        
        # Generate pattern with enhanced smoothing for improved gaits
        pattern = make_kinematic_ref(cos_wave, step_k=params['step_k'], 
                                     scale=params['scale'], dt=dt,
                                     smooth=True, filter_strength=0.2)
        
        # Calculate number of cycles needed for the duration
        n_cycles = (total_physics_steps_per_gait // len(pattern)) + 1
        reference_positions = np.tile(pattern, (n_cycles, 1))[:total_physics_steps_per_gait]
        
        # Create video frames
        frames = []
        for i in range(total_physics_steps_per_gait):
            # Get current joint positions and velocities
            current_positions = data.qpos[7:19]  # Assuming joints start at index 7
            current_velocities = data.qvel[6:18]  # Assuming joint velocities start at index 6
            
            # Get reference positions for current step
            targets = reference_positions[i]
            
            # Compute control signals using controller
            control_signals = controller.compute_control(targets, current_positions, current_velocities)
            
            # Apply control signals to actuators
            data.ctrl[:] = control_signals
            
            # Step physics
            mujoco.mj_step(model, data)
            
            # Only render at specified intervals to maintain frame rate
            if i % physics_steps_per_frame == 0:
                # Update scene with data and render (using free camera)
                renderer.update_scene(data, camera=-1)  # Use free camera (-1) instead of camera ID 0
                pixels = renderer.render()
                frames.append(pixels.copy())  # Make sure to copy to avoid reference issues
                
                if len(frames) % 20 == 0:
                    print(f"  Rendered {len(frames)} frames for {params['name']}")
        
        all_videos.append((params['name'], frames))
        print(f"Completed {params['name']} gait with {len(frames)} frames")
    
    return all_videos

## Generate and Display Different Gaits with Improved Control
Let's try generating and displaying different gaits with our smoother controller.

In [7]:
# Reset simulation fully
mujoco.mj_resetData(model, data)

# Define smoother trajectory interpolation function
def smooth_interpolation(start, end, t, smoothing_factor=3.0):
    """
    Performs a smooth interpolation between start and end positions.
    Uses a higher-order polynomial function for smoother transitions.
    """
    # Smoothstep function (higher order for smoother transitions)
    if smoothing_factor == 3.0:
        factor = t * t * (3.0 - 2.0 * t)
    elif smoothing_factor == 5.0:
        factor = t * t * t * (t * (t * 6.0 - 15.0) + 10.0)
    else:
        factor = t * t * (3.0 - 2.0 * t)
        
    return start + factor * (end - start)

# Define a low-pass filter for smoothing control signals
def low_pass_filter(new_value, old_value, alpha=0.1):  # Default alpha for moderate smoothing
    """
    Simple first-order low-pass filter. Lower alpha means more smoothing.
    """
    return alpha * new_value + (1 - alpha) * old_value

# Create renderer with resolution within framebuffer limits
height, width = 480, 640
if 'renderer' in globals():
    del renderer
renderer = mujoco.Renderer(model, height=height, width=width)
video_frames = []

# Simulation parameters - extended for longer observation
fps = 30
simulation_time = 12.0  # Extended to 12 seconds to show stability
total_physics_steps = int(simulation_time / model.opt.timestep)
physics_steps_per_frame = max(1, int(1.0 / (fps * model.opt.timestep)))
n_frames = total_physics_steps // physics_steps_per_frame

print(f"Physics timestep: {model.opt.timestep:.4f} s, Total steps: {total_physics_steps}, Frames: {n_frames}")
print(f"Available actuators: {[model.actuator(i).name for i in range(model.nu)]}")

# IMPROVED CONTROLLER CONFIGURATION
# Higher position gain for precise posture holding, balanced damping for stability
controller = ImprovedPDController(kp=180.0, kd=22.0, kff=15.0, max_velocity=4.0)
controller.set_dt(model.opt.timestep)
controller.reset()

# Define a stable stance posture for standing
stable_stance = np.zeros(12)

# Wider stance with outward hip angles for stability
stable_stance[0] = 0.05   # LF_HAA - outward angle
stable_stance[3] = -0.05  # RF_HAA - outward angle
stable_stance[6] = 0.05   # LH_HAA - outward angle
stable_stance[9] = -0.05  # RH_HAA - outward angle

# Lower center of mass with bent knees for stability
stable_stance[2] = 0.3    # LF_KFE - increased knee bend
stable_stance[5] = 0.3    # RF_KFE - increased knee bend
stable_stance[8] = 0.3    # LH_KFE - increased knee bend
stable_stance[11] = 0.3   # RH_KFE - increased knee bend

# Add slight offset to hip flexion/extension for weight distribution
stable_stance[1] = 0.05   # LF_HFE - slight forward lean
stable_stance[4] = 0.05   # RF_HFE - slight forward lean
stable_stance[7] = -0.05  # LH_HFE - slight backward lean
stable_stance[10] = -0.05 # RH_HFE - slight backward lean

# Create reference positions with subtle random variations
# to simulate slight "breathing" movement while standing
reference_positions = np.tile(stable_stance, (total_physics_steps, 1))

# Add very subtle random movement to simulate natural standing motion
np.random.seed(42)  # For reproducible results
for i in range(total_physics_steps):
    # Only add tiny variations every 100 steps (slow changes)
    if i % 200 == 0:
        # Tiny random variation (max ±0.003 radians)
        tiny_variation = (np.random.random(12) - 0.5) * 0.006
        reference_positions[i:i+200] += tiny_variation.reshape(1, 12)

# Smooth these tiny variations
for joint in range(12):
    reference_positions[:, joint] = filter_signal(reference_positions[:, joint], alpha=0.02)

# Get initial pose
initial_pose = data.qpos[7:19].copy()

# Moderately paced transition from current pose to stable stance
transition_steps = int(2.0 / model.opt.timestep)  # 2 seconds for transition
for i in range(min(transition_steps, total_physics_steps)):
    # Normalized time from 0 to 1
    progress = i / transition_steps
    # Use 5th order smoothstep for very smooth transitions
    reference_positions[i] = smooth_interpolation(initial_pose, reference_positions[i], progress, 5.0)

# Data collection arrays
position_history = []
velocity_history = []
control_history = []
com_positions = []  # For tracking center of mass

# Balance regulation strength - increased for better stability
balance_gain = 0.7  # Higher gain for more responsive balance correction

# Run simulation with improved motion control
frame_count = 0
prev_control = np.zeros(12)  # For control smoothing
filtered_control = np.zeros(12)  # For additional filtering

# Define quaternion to RPY conversion function
def compute_rpy_from_quaternion(q):
    """Convert quaternion to roll, pitch, yaw angles."""
    w, x, y, z = q
    
    # Roll (x-axis rotation)
    sinr_cosp = 2.0 * (w * x + y * z)
    cosr_cosp = 1.0 - 2.0 * (x * x + y * y)
    roll = np.arctan2(sinr_cosp, cosr_cosp)
    
    # Pitch (y-axis rotation)
    sinp = 2.0 * (w * y - z * x)
    if abs(sinp) >= 1:
        pitch = np.copysign(np.pi / 2, sinp)
    else:
        pitch = np.arcsin(sinp)
        
    # Yaw (z-axis rotation)
    siny_cosp = 2.0 * (w * z + x * y)
    cosy_cosp = 1.0 - 2.0 * (y * y + z * z)
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    
    return np.array([roll, pitch, yaw])

for i in range(total_physics_steps):
    # Get current joint positions and velocities
    current_positions = data.qpos[7:19]
    current_velocities = data.qvel[6:18]
    
    # Get reference position for current step
    targets = reference_positions[i]
    
    # Get body orientation
    orientation = data.qpos[3:7]  # Quaternion for base orientation
    
    # Store center of mass position
    base_position = data.qpos[:3]
    com_positions.append(base_position.copy())
    
    # Compute balance adjustment with improved responsiveness
    roll, pitch = compute_rpy_from_quaternion(orientation)[0:2]
    
    # Apply more responsive stiffness adjustment
    tilt_amount = np.sqrt(roll**2 + pitch**2)
    stiffness_factor = 1.0 + balance_gain * tilt_amount
    
    # Apply orientation adjustment with moderate response
    if i > 0:
        # Moderately responsive gain adjustment
        controller.kp = low_pass_filter(180.0 * stiffness_factor, controller.kp, 0.1)
    else:
        controller.kp = 180.0 * stiffness_factor
    
    # Compute control signals
    control_signals = controller.compute_control(targets, current_positions, current_velocities)
    
    # More responsive but still smooth rate limiting
    if i > 0:
        # Moderate rate limiting for smooth yet responsive transitions
        max_change = 0.5  # Balanced limit for controlled transitions
        control_signals = np.clip(control_signals, prev_control - max_change, prev_control + max_change)
        
        # Moderate low-pass filtering for balance between smoothness and responsiveness
        filtered_control = low_pass_filter(control_signals, filtered_control, 0.2)
        control_signals = filtered_control
    else:
        filtered_control = control_signals.copy()
    
    # Store for next iteration
    prev_control = control_signals.copy()
    
    # Apply control signals to actuators
    data.ctrl[:] = control_signals
    
    # Store data for analysis
    position_history.append(current_positions.copy())
    velocity_history.append(current_velocities.copy())
    control_history.append(control_signals.copy())
    
    # Step the simulation
    mujoco.mj_step(model, data)
    
    # Only render every physics_steps_per_frame steps
    if i % physics_steps_per_frame == 0:
        renderer.update_scene(data, camera=-1)
        pixels = renderer.render()
        video_frames.append(pixels.copy())
        frame_count += 1
        
        if frame_count % 20 == 0:
            print(f"Rendered {frame_count}/{n_frames} frames")

print(f"Total frames rendered: {len(video_frames)}")

# Display the video
media.show_video(video_frames, fps=fps)

print("Balanced standing simulation complete!")
print(f"Max joint velocity: {np.max(np.abs(velocity_history)):.4f} rad/s")

Physics timestep: 0.0020 s, Total steps: 6000, Frames: 375
Available actuators: ['LF_HAA', 'LF_HFE', 'LF_KFE', 'RF_HAA', 'RF_HFE', 'RF_KFE', 'LH_HAA', 'LH_HFE', 'LH_KFE', 'RH_HAA', 'RH_HFE', 'RH_KFE']
Rendered 20/375 frames
Rendered 40/375 frames
Rendered 60/375 frames
Rendered 80/375 frames
Rendered 100/375 frames
Rendered 120/375 frames
Rendered 140/375 frames
Rendered 160/375 frames
Rendered 180/375 frames
Rendered 200/375 frames
Rendered 220/375 frames
Rendered 240/375 frames
Rendered 260/375 frames
Rendered 280/375 frames
Rendered 300/375 frames
Rendered 320/375 frames
Rendered 340/375 frames
Rendered 360/375 frames
Total frames rendered: 375


This browser does not support the video tag.


Balanced standing simulation complete!
Max joint velocity: 52.7783 rad/s
